- Algorithm description
- Algorithm code
- Empirical and canonical test dataset
- Performance measures (analytical and empirical)
- Evaluation
- Conclusion


                                                                                                                        © Moon

# Project Report: Evaluation of the Centrality Algorithm, PageRank

## Introduction

Centrality algorithms are one of the categories of graph algorithms. They identify the important nodes in a given graph and those nodes are defined as vertices with many direct or indirect connections.  One of the centrality algorithms is called $PageRank(PR)$ which identify most important vertices of a graph by measuring the direct influence of nodes based on proportional rank. Now days, developers utilize on analytics, web, social networks, and etc. PR can be used to identify influencers in social media for example or identify potential attack targets in a network. Google rank websites in their Search engine results. 


## The Algorithm

The PR ranks depend on the number of pages, damping factor, and a number of iterations. The PageRank relies on an arbitrary probability distribution in which a person randomly clicks on links will arrive at any particular page. The probability which a person independently will continue is a damping fator d. PR computations require iterations through a number of pages to adjust approximate PR values to the theoretical value.

The iteration equation of the page rank value of  𝑖  is given by

**PR(n) = (1-d)/N + d*(PR(n1)/num_neighbors(n1) + ... + PR(n_last)/num_neighbors(n_last))**

where the damping factor $d=D$, $\frac{d}{n}$ denotes random walk score, $OutputDegree(P_j)$ denote how many pages are linked as children pages for the page $j$.